In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movies = pd.read_csv('/content/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/tmdb_5000_credits.csv')

In [ ]:

movies.shape

(4803, 20)

In [ ]:
movies = movies[['id','title', 'overview', 'genres', 'keywords']]
movies.head(1)

,id,title,overview,genres,keywords
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."


In [ ]:
#genres, id, keywords, tittle, overview, 
credits.shape

(4803, 4)

In [ ]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [ ]:
movies = movies.merge(credits, on='title')
movies.shape

(1832, 8)

In [ ]:
#dropping duplicates columns
movies = movies.drop_duplicates()
movies.head(1)

,id,title,overview,genres,keywords,movie_id,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [ ]:
#Checking is there any missing data
movies.isnull().sum()
# 3 missing datas found

id          0
title       0
overview    3
genres      0
keywords    0
movie_id    0
cast        0
crew        0
dtype: int64

In [ ]:
#droping the missing data rows
movies.dropna(inplace=True)
movies.shape

(1831, 8)

In [ ]:
#Checking is there any duplicated data
movies.duplicated().sum()
#no duplicated data found

0

In [ ]:
#function that will convert string of list to list
import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
      L.append(i['name'])
    return L;

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies.iloc[0].keywords

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies.iloc[0].cast

In [ ]:
#function that will take top 3 actor of a movie from the cast
def convert3(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
      L.append(i['name'])
      count+=1
      if count==3:
        break
    return L;

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
movies.iloc[0].crew

In [ ]:
#function that will take only director name
def fetchDirector(obj):
    L = []
    for i in ast.literal_eval(obj):
      if i['job']=='Director':
        L.append(i['name'])
        break
    return L;

In [ ]:
movies['crew'] = movies['crew'].apply(fetchDirector)

In [ ]:
movies.head(1)

,id,title,overview,genres,keywords,movie_id,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [ ]:
#converting overview column from string to list
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
#joining two words for eg. science fiction --> sciencefiction
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head(1)

,id,title,overview,genres,keywords,movie_id,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [ ]:
#concatenate text colums into a single column
movies['text'] = movies['genres'] + movies['overview'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
df = movies[['id','title','text']]

In [ ]:
#making lists of text's column again into string
df['text'] = df['text'].apply(lambda x:" ".join(x))

In [ ]:
#converting to lower case
df['text'] = df['text'].apply(lambda x:x.lower())

In [ ]:
#Bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(df['text']).toarray()

In [ ]:
#stemming
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  
  return " ".join(y)

In [ ]:
df['text'] = df['text'].apply(stem)

In [ ]:
df

In [ ]:
#Bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(df['text']).toarray()

In [ ]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zooey', 'zooeydeschanel', 'zookeep'],
      dtype=object)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [ ]:
similarity[1]

array([0.07610194, 1.        , 0.05812382, ..., 0.04433511, 0.05905386,
       0.03001501])

In [ ]:
def recommend(movie):
  movie_index = df[df['title']==movie].index[0]
  distance = similarity[movie_index]
  movies_list = sorted(enumerate(distance),reverse=True,key=lambda x:x[1])[1:6]

  for i in movies_list:
    print(df.iloc[i[0]].title)

In [ ]:
recommend('The Dark Knight')

The Dark Knight Rises
Batman Begins
Batman Returns
Batman Forever
Batman
